# Práctica Guiada - Técnicas para datasets desbalanceados

El problema de los datasets desbalanceados se encuentra en una amplia variedad de problemas de Machine Learning como la detección de comentarios hechos a través de bots, el diagnóstico de enfermedades o la búsqueda de transacciones fraudulentas en un sistema.

Existen dos grandes enfoques para abordar datasets que se encuentran desbalanceados:

1.  Hacer un resampling de la muestra, para entrenar al algoritmo con proporciones similares
2.  Incorporar el desbalance a la función de costos del algoritmo para que tenga incentivos a elegir los parámetros que mejor discriminan la clase minoritaria.

## Clasificación sobre datos desbalanceados

A continuación presentamos un dataset de la empresa americana Lending Club, que se dedica a proveer servicios financieros para distintos segmentos. 

A continuación vamos a utilizar información abierta del portal de la empresa para intentar predecir cuáles de los créditos terminan en default. Para más información sobre el datset pueden ingresar <a href='https://www.lendingclub.com/info/download-data.action'> aquí </a>

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [6]:
loans = pd.read_csv('../Data/loans.csv',low_memory=False)

## Análisis exploratorio

#### Balanceo de la clase

La clase que vamos a intentar predecir es "bad loans" que indica si el préstamo fue pagado a tiempo o no. Observamos que la clase está desbalanceada, la mayoría de los préstamos se pagan a tiempo.

In [7]:
loans['bad_loans'].value_counts(normalize=True)

0    0.811185
1    0.188815
Name: bad_loans, dtype: float64

#### Variables numéricas
Podemos buscar las variables numéricas relevantes observando cuáles son las que tienen mayor correlación con la clase, bad_loans.

In [8]:
loans.corr()['bad_loans'].apply(abs).sort_values(ascending=False)

bad_loans                1.000000
int_rate                 0.227545
revol_util               0.112548
dti                      0.107652
total_rec_late_fee       0.032719
payment_inc_ratio        0.025879
short_emp                0.019498
emp_length_num           0.008342
last_delinq_none         0.003466
last_major_derog_none    0.002390
Name: bad_loans, dtype: float64

#### Variables categóricas

Para hacer el mismo análisis sobre variables categóricas, hacemos un group by por categoría para ver diferencias relevantes en la cantidad de créditos con default.

In [9]:
loans.groupby('home_ownership')['bad_loans'].mean().sort_values()

home_ownership
MORTGAGE    0.169142
OWN         0.194609
RENT        0.209484
OTHER       0.229050
Name: bad_loans, dtype: float64

In [10]:
loans.groupby('term')['bad_loans'].mean().sort_values()

term
 36 months    0.159661
 60 months    0.303757
Name: bad_loans, dtype: float64

In [11]:
loans.groupby('purpose')['bad_loans'].mean().sort_values()

purpose
car                   0.130526
major_purchase        0.141346
wedding               0.152031
home_improvement      0.159088
credit_card           0.159365
house                 0.180100
vacation              0.187209
debt_consolidation    0.195932
medical               0.206596
moving                0.210169
other                 0.220425
small_business        0.322304
Name: bad_loans, dtype: float64

#### Valores faltantes

Inspeccionamos los valores faltantes

In [12]:
loans.isnull().sum()

grade                    0
home_ownership           0
emp_length_num           0
sub_grade                0
short_emp                0
dti                      0
term                     0
purpose                  0
int_rate                 0
last_delinq_none         0
last_major_derog_none    0
revol_util               0
total_rec_late_fee       0
payment_inc_ratio        4
bad_loans                0
dtype: int64

In [13]:
# Notamos que la variable payment_inc_ratio tiene sólo 4 valores nulos y mustra una correlación importante con la clase.
# Además sabemos que la relación cuota/ingreso es importante a al hora de evaluar la capacidad de repago. 
# Eliminamos, entonces, los valores con payment_inc_ratio desconocido.
loans = loans[loans.payment_inc_ratio.notnull()].copy()

In [14]:
loans.sample(10)

,grade,home_ownership,emp_length_num,sub_grade,short_emp,dti,term,purpose,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,bad_loans
86128,A,MORTGAGE,11,A1,0,11.27,36 months,major_purchase,6.03,1,1,20.8,0.0,1.31224,0
11731,E,MORTGAGE,3,E2,0,21.91,60 months,debt_consolidation,18.39,1,1,66.3,0.0,9.89327,0
23924,A,MORTGAGE,11,A5,0,5.63,36 months,house,7.88,1,1,30.5,0.0,2.50252,0
34536,A,MORTGAGE,1,A4,1,5.49,36 months,home_improvement,8.32,0,1,1.2,14997.0,4.27286,1
56227,D,RENT,2,D1,0,3.89,36 months,debt_consolidation,17.56,1,1,83.5,0.0,4.45573,0
104261,A,RENT,2,A5,0,4.50,36 months,credit_card,8.90,0,1,25.1,0.0,7.93850,0
52761,E,MORTGAGE,8,E5,0,27.75,60 months,debt_consolidation,22.70,1,1,88.0,0.0,2.50667,1
38067,B,RENT,4,B3,0,0.82,36 months,credit_card,9.33,0,1,0.0,0.0,6.08648,0
22885,D,MORTGAGE,6,D2,0,17.04,36 months,debt_consolidation,15.21,0,1,27.1,0.0,10.43060,0
29757,A,RENT,8,A5,0,20.24,36 months,credit_card,8.94,1,1,17.3,0.0,4.56401,0


## Modelo Predictivo: regresión logística

Vamos a utilizar Regresión Logística para predecir la clase bad_loans. 

In [15]:
# Generamos las variables dummies para los datos categóricos.
loans_enconded = pd.get_dummies(loans)

In [16]:
training_features, test_features, \
training_target, test_target, = train_test_split(loans_enconded.drop(['bad_loans'], axis=1),
                                               loans_enconded['bad_loans'],
                                               test_size = .1,
                                               random_state=12)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report


In [18]:
model = LogisticRegression(C=1e10)
model.fit(training_features,training_target)

LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [19]:
y_pred = model.predict(test_features)

In [20]:
roc_auc_score(test_target,y_pred)

0.51097885675688404

In [21]:
print(classification_report(test_target,y_pred))

             precision    recall  f1-score   support

          0       0.82      0.99      0.90      9960
          1       0.54      0.03      0.05      2301

avg / total       0.76      0.81      0.74     12261



Cuando evaluamos el modelo sobre datos no observados, el área debajo de la curva es muy cercana a 0.5. Especificamente, el recall a la hora de encontrar los "malos créditos" es  muy malo. Uno de los problemas que tenemos para hacer un buen modelo es el desbalanceo de los datos. 

Vamos a intentar dos posibles soluciones.

## Oversampling

Para aumentar la representación de la clase minoritaria vamos a hacer un oversampling utilizando el algoritmo SMOTE (Synthetic Minority Oversample) del paquete imblearn.

Este algoritmo genera nuevos datos utilizando la técnica de los K vecinos más cercanos.
Para generar un nuevo punto:

1. Se elige un punto al azar de la clase minoritaria y sus K vecinos más cercanos.
2. Se elige al azar uno de esos vecinos.
3. Se calcula el vector entre el punto seleccionado y el vecino seleccionado al azar y se lo multiplica por un número aleatorio entre 0 y 1.
4. El punto aleatorio dentro del vector es el nuevo dato para el oversampling.


In [29]:
len(training_features), len(training_target)

(110342, 110342)

In [22]:
sm = SMOTE(random_state=12)
x_train_res, y_train_res = sm.fit_sample(training_features, training_target)

In [31]:
len(x_train_res), len(y_train_res)

(178986, 178986)

In [23]:
model.fit(x_train_res,y_train_res)

LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [24]:
y_pred = model.predict(test_features)

In [25]:
roc_auc_score(test_target,y_pred)

0.6335922569024468

In [26]:
print(classification_report(test_target,y_pred))

             precision    recall  f1-score   support

          0       0.88      0.61      0.73      9960
          1       0.28      0.65      0.39      2301

avg / total       0.77      0.62      0.66     12261



# Class Weights

La otra técnica que podemos utilizar para corregir el desbalance de los datos es incorporar en la función de costos del algoritmo un mayor peso para los errores de entrenamiento cometidos sobre los puntos de la clase minoritaria.

<strong> Ejercicio: Implementar la Regresión Logística con un parámetro que cambie la ponderación de las clases. <br />
¿Cómo es la performance del nuevo modelo con respecto a la regresión logística original? <br />
¿Tiene sentido aplicar esta corrección junto con el algoritmo SMOTE?
<strong />

In [35]:
model_balanced = LogisticRegression(C=1e10, class_weight='balanced')
model_balanced.fit(training_features, training_target)
y_balanced_prediction = model_balanced.predict(test_features)
print(roc_auc_score(test_target, y_balanced_prediction))
print(classification_report(test_target, y_balanced_prediction))

0.633227957462
             precision    recall  f1-score   support

          0       0.88      0.62      0.73      9960
          1       0.28      0.65      0.39      2301

avg / total       0.77      0.63      0.67     12261

